In [2]:
from http import client
from urllib import response
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from img2vec_pytorch import Img2Vec
from sentence_transformers import SentenceTransformer
import plotly.express as px
import plotly.graph_objects as go
import urllib.request
import requests
import pandas as pd
import json
import pprint
import seaborn as sns
import matplotlib.pyplot as plt
import json
import numpy as np

/home/dustin/anaconda3/envs/web3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
description_list = []
contract_addresses = []
name_list = []

for j in range(30):
  with open(f'/home/dustin/Documents/Ducia/NFT_image_recommendation/Blockdaemon_NFT_Contracts/{j+1}.json') as json_file:
      json_data = json.load(json_file)['data']

  address_list = []
  for i in json_data:
      if ("contracts" in i.keys() and len(i["contracts"]) > 0 and "address" in i["contracts"][0].keys()):
          address = i["contracts"][0]["address"]
          address_list.append(address)

  print("new file yay :)")

  counter = 0
  for address in address_list:
      counter += 1
      #print(counter)
      # evtl muessen wir checken ob alles erc721 sind und tokenID 1 immer existiert
      URL = f"https://eth-mainnet.alchemyapi.io/v2/demo/getNFTMetadata?contractAddress={address}&tokenId=1&tokenType=erc721"
      NFT = requests.get(URL)
      try:
          NFTjson = NFT.json()
      except:
          continue
      if (type(NFTjson) is dict and "metadata" in NFTjson.keys() and type(NFTjson["metadata"]) is dict and "description" in NFTjson["metadata"].keys()):
          print('This is what we want')
          external_url = NFTjson["metadata"]["description"]
          description_list.append(external_url)
          contract_addresses.append(address)
          name = "Not Available"
          if (type(NFTjson["metadata"]) is dict and "title" in NFTjson["metadata"].keys()):
            name = NFTjson["metadata"]["title"]
          name_list.append(name)


In [3]:
#np.savez('/home/dustin/Documents/Ducia/NFT_image_recommendation/NFT_attributes.npz', description_list=description_list, contract_addresses=contract_addresses, name_list=name_list)
NFT_attributes = np.load('./data/NFT_attributes.npz')
d = dict(zip(("description_list","contract_addresses","name_list"), (NFT_attributes[k] for k in NFT_attributes)))
description_list = d['description_list']
contract_addresses = d['contract_addresses']
name_list = d['name_list']

In [3]:
#print('Loading Model')
modelPath = "./data/bert-base-model"
print('Loading Model')
model = SentenceTransformer(modelPath)

#description embeddings
print(f'Calculating embeddings for {len(description_list)} descriptions')
desc_embeddings = model.encode(description_list, normalize_embeddings=True, show_progress_bar=True, batch_size = 64)

#desc_embeddings

Loading Model


NameError: name 'description_list' is not defined

In [4]:
#np.savez('/home/dustin/Documents/Ducia/NFT_image_recommendation/desc_emb.npz', desc_embeddings=desc_embeddings)
desc_emb = np.load('./data/desc_emb.npz')
desc_embeddings = dict(zip(("desc_embeddings"), (desc_emb[k] for k in desc_emb)))['d']
print(desc_embeddings.shape)

(11083, 768)


In [26]:
#Test
desc_embeddings = desc_embeddings[:1000]
description_list = description_list[:1000]
contract_addresses = contract_addresses[:1000]
name_list = name_list[:1000]

In [ ]:
nft_tsne = TSNE(n_components=3, n_jobs = -1).fit_transform(desc_embeddings)
nft_tsne_df = pd.DataFrame(nft_tsne)
nft_tsne_df = (nft_tsne_df-nft_tsne_df.mean())/nft_tsne_df.std()
nft_tsne_df

In [6]:
truncated_descriptions = [desc[0:50] + '...' if len(desc) > 50 else desc for desc in description_list]
final_df = pd.concat([nft_tsne_df, pd.DataFrame(contract_addresses), pd.DataFrame(name_list), pd.DataFrame(truncated_descriptions)], axis = 1)
final_df.columns = ['X', 'Y', 'Z', 'NFT_Address', 'Name', 'D']
final_df

NameError: name 'nft_tsne_df' is not defined

In [ ]:
fig = px.scatter_3d(
    final_df, x= 'X', y= 'Y', z= 'Z',
    size=[1]*len(final_df),
    color = 'NFT_Address', 
    labels= {'influence_rating': 'Legend'},
    hover_data = 'D',
)
fig.show()

In [ ]:
fig.update_layout(xaxis=dict(showspikes=False) , 
        width=2000,
        height=1000,
        margin=dict(
            l=2,
            r=2,
            b=2,
            t=2,
            pad=4
        ), )

fig.update_traces(marker_size = 3 )
fig.update_coloraxes(showscale=False)
fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False )

fig.write_html("attributes+description_pca.html")
fig.show()

In [18]:
# Testing Random Neighborhood
index = np.random.randint(0, len(desc_embeddings))
sample = desc_embeddings[index]
min_index = 0
min_distance = 10

for i, di in enumerate(desc_embeddings):
    distance = np.sum(np.square((sample - di)))
    if distance < min_distance and index != i:
        min_distance = distance
        min_index = i

print(f'The closest NFT is {min_index} with a distance of {min_distance}')
print(f'\ndescription {min_index}: {description_list[min_index]}')
print(f'\ndescription {index}: {description_list[index]}')


The closest NFT is 688 with a distance of 0.27132129669189453

description 688: alien enemes are a collection of 10,000 randomly generated aliens provides you not only a mind-blowing PFP and also membership in our exclusive club for meets, events, exclusive merch, 2D Comic Books and more roadmap versions will be released as we progress!

description 6725: SCAR is a collection of 10,200 utility-enabled NFTs. Hold Aliens and Enjoy a variety of benefits!
